In [13]:
import torch
import torch_sparse
import numpy as np
from rgnn_at_scale.aggregation import (_sparse_top_k, soft_weighted_medoid, soft_weighted_medoid_k_neighborhood,
                                       weighted_dimwise_median, weighted_medoid, weighted_medoid_k_neighborhood)
from rgnn_at_scale.data import prep_graph

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
device = 0 if torch.cuda.is_available() else 'cpu'
temperature = 1e-3
device

0

In [4]:
A = torch.tensor([[0.5, 0.3, 0, 0.4], [0.3, 0.2, 0, 0], [0, 0, 0.9, 0.3],
                          [0.4, 0, 0.4, 0.4]], dtype=torch.float32).to(device)
x = torch.tensor([[-10, 10, 10], [-1, 1, 1], [0, 0, 0], [10, -10, -10]], dtype=torch.float32).to(device)

A_sparse_tensor = torch_sparse.SparseTensor.from_dense(A).to(device)


/nfs/homedirs/schmidtt/miniconda3/lib/python3.8/site-packages/torch_sparse/tensor.py:46: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /opt/conda/conda-bld/pytorch_1595629395347/work/torch/csrc/utils/python_arg_parser.cpp:766.)
  index = mat.nonzero()


In [10]:
A_sparse_tensor = torch_sparse.SparseTensor.from_dense(torch.load("datasets/test_adj_matrix.pt"))
#x = torch_sparse.SparseTensor.from_dense(torch.load("datasets/test_attr_matrix.pt"))
x = torch.load("datasets/test_attr_matrix.pt")
A_sparse_tensor, x

(SparseTensor(row=tensor([   0,    0,    0,  ..., 2993, 2993, 2993]),
              col=tensor([1636, 1638, 2357,  ...,  200,  745, 1865]),
              val=tensor([1., 1., 1.,  ..., 1., 1., 1.]),
              size=(2995, 2995), nnz=8416, density=0.09%),
 tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]))

In [20]:
graph = prep_graph("cora_ml", device)
attr, adj, labels = graph[:3]
A_sparse_tensor = torch_sparse.SparseTensor.from_torch_sparse_coo_tensor(adj).to(device)
x = attr
A_sparse_tensor, x

(SparseTensor(row=tensor([   0,    0,    0,  ..., 2808, 2809, 2809], device='cuda:0'),
              col=tensor([   0, 1579, 1581,  ..., 2808, 1399, 2809], device='cuda:0'),
              val=tensor([1., 1., 1.,  ..., 1., 1., 1.], device='cuda:0'),
              size=(2810, 2810), nnz=18772, density=0.24%),
 tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0'))

In [21]:
 medoids = soft_weighted_medoid_k_neighborhood(A_sparse_tensor,
                                                      x,
                                                      k=2,
                                                      temperature=temperature,
                                                      threshold_for_dense_if_cpu=0)

_sparse_top_k
is cuda
tensor([[   0, 1579],
        [   1,    2],
        [   1,    2],
        ...,
        [ 639, 2216],
        [ 198,  730],
        [1399, 2809]], device='cuda:0')
torch.Size([2810, 2])
tensor(2809, device='cuda:0')


In [9]:
 medoids = soft_weighted_medoid_k_neighborhood(A_sparse_tensor,
                                                      x,
                                                      k=2,
                                                      temperature=temperature,
                                                      threshold_for_dense_if_cpu=0)

_sparse_top_k
is cuda
tensor([[1., 1.],
        [1., 1.],
        [1., 1.],
        ...,
        [1., 1.],
        [1., 1.],
        [0., 0.]], device='cuda:0')
tensor([[1636, 1638],
        [1224, 1955],
        [   1, 1076],
        ...,
        [ 653, 2332],
        [ 200,  745],
        [  -1,   -1]], device='cuda:0')
tensor(2988, device='cuda:0')


RuntimeError: The size of tensor a (2994) must match the size of tensor b (2995) at non-singleton dimension 0

In [ ]:
medoids

In [8]:
row_sum = A.sum(-1)
layer_idx = 0
assert torch.all(medoids[layer_idx] == row_sum[layer_idx] * x[0])

layer_idx = 1
assert torch.all(medoids[layer_idx] == row_sum[layer_idx] * x[0])

layer_idx = 2
assert torch.all(medoids[layer_idx] == row_sum[layer_idx] * x[2])

layer_idx = 3
assert (torch.all(medoids[layer_idx] == row_sum[layer_idx] * (x[0] + x[2]) / 2)
        or torch.all(medoids[layer_idx] == row_sum[layer_idx] * (x[0] + x[3]) / 2)
        or torch.all(medoids[layer_idx] == row_sum[layer_idx] * (x[2] + x[3]) / 2))

For debugging _sparse_top_k:

In [13]:
unroll_idx = np.array([0, 1, 0, 1, 0, 1, 0, 1])
new_idx = torch.tensor([[0, 0, 1, 1, 2, 2, 3, 3],
                        [0, 3, 0, 1, 2, 3, 0, 2]])
value_idx = torch.tensor([0, 2, 3, 4, 5, 6, 7, 8])
values = torch.tensor([[0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.]])